In [14]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
import pandas as pd

In [4]:
from nba_api.stats.static import teams

nba_teams = teams.get_teams()
# Select the dictionary for the Celtics, which contains their team ID
celtics = [team for team in nba_teams if team['abbreviation'] == 'BOS'][0]
celtics_id = celtics['id']

In [6]:
from nba_api.stats.endpoints import leaguegamefinder

# Query for games where the Celtics were playing
gamefinder = leaguegamefinder.LeagueGameFinder(team_id_nullable=celtics_id)
# The first DataFrame of those returned is what we want.
games = gamefinder.get_data_frames()[0]
games

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS
0,22024,1610612738,BOS,Boston Celtics,0022400202,2024-11-10,BOS @ MIL,W,240,113,...,0.889,8,34,42,27,5,2,12,16,6.0
1,22024,1610612738,BOS,Boston Celtics,0022400187,2024-11-08,BOS vs. BKN,W,266,108,...,0.842,4,35,39,23,7,9,13,15,4.0
2,22024,1610612738,BOS,Boston Celtics,0022400172,2024-11-06,BOS vs. GSW,L,240,112,...,0.810,13,34,47,22,5,3,12,18,-6.0
3,22024,1610612738,BOS,Boston Celtics,0022400157,2024-11-04,BOS @ ATL,W,239,123,...,0.692,11,39,50,27,15,7,12,14,30.0
4,22024,1610612738,BOS,Boston Celtics,0022400141,2024-11-02,BOS @ CHA,W,240,113,...,0.893,8,36,44,23,4,4,15,17,10.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3928,21983,1610612738,BOS,Boston Celtics,0028300052,1983-11-05,BOS @ WAS,W,240,120,...,0.778,5,28,33,36,7,7,16,26,NaN
3929,21983,1610612738,BOS,Boston Celtics,0028300043,1983-11-04,BOS vs. IND,W,240,121,...,0.841,11,34,45,28,8,5,22,26,NaN
3930,21983,1610612738,BOS,Boston Celtics,0028300033,1983-11-02,BOS vs. MIL,W,240,119,...,0.816,14,24,38,26,7,7,18,18,NaN
3931,21983,1610612738,BOS,Boston Celtics,0028300019,1983-10-29,BOS @ CLE,W,240,108,...,0.690,13,31,44,25,12,7,13,22,NaN


In [7]:
games.columns

Index(['SEASON_ID', 'TEAM_ID', 'TEAM_ABBREVIATION', 'TEAM_NAME', 'GAME_ID',
       'GAME_DATE', 'MATCHUP', 'WL', 'MIN', 'PTS', 'FGM', 'FGA', 'FG_PCT',
       'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'FT_PCT', 'OREB', 'DREB',
       'REB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PLUS_MINUS'],
      dtype='object')

In [9]:
games.MATCHUP

0         BOS @ MIL
1       BOS vs. BKN
2       BOS vs. GSW
3         BOS @ ATL
4         BOS @ CHA
           ...     
3928      BOS @ WAS
3929    BOS vs. IND
3930    BOS vs. MIL
3931      BOS @ CLE
3932      BOS @ DET
Name: MATCHUP, Length: 3933, dtype: object

In [17]:
games_2024 = games[(games['GAME_DATE'] >= '2023-10-24') & (games['GAME_DATE'] <= '2024-04-14')]
#games_2024.to_excel("games_2024.xlsx", index=False)

## Tratamento da base

In [22]:
X = games[['MIN', 'FGM','FGA', 'FG_PCT',
       'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'FT_PCT', 'OREB', 'DREB',
       'REB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PLUS_MINUS'
]]  # Variáveis independentes
y = games['PTS']  # Variável dependente (valor que queremos prever) 
# Dividir os dados em treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [23]:
# Criar o modelo Random Forest Regressor
model = RandomForestRegressor(n_estimators=100, random_state=42)

# Treinar o modelo
model.fit(X_train, y_train)

RandomForestRegressor(random_state=42)

In [24]:
# Fazer previsões
y_pred = model.predict(X_test)

In [25]:
# Avaliar o modelo
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)
mae = mean_absolute_error(y_test, y_pred)
print("Mean Absolute Error (MAE):", mae)

Mean Squared Error: 5.264684600451785
Mean Absolute Error (MAE): 1.1140046590427786
